<a href="https://colab.research.google.com/github/yunju-1118/ESAA/blob/OB/09_12_%E1%84%89%E1%85%A6%E1%84%89%E1%85%A7%E1%86%AB_%E1%84%86%E1%85%A9%E1%84%83%E1%85%A6%E1%86%AF%E1%84%92%E1%85%AE%E1%86%AB%E1%84%85%E1%85%A7%E1%86%AB_%E1%84%8B%E1%85%A7%E1%86%AB%E1%84%89%E1%85%B3%E1%86%B8%E1%84%86%E1%85%AE%E1%86%AB%E1%84%8C%E1%85%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___

=> 확률적 경사하강법 / 미니 배치 경사하강법을 사용하여 빠르게 fitting 할 수 있다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

=> 데이터에 과적합 되었기 때문이다. sample size를 늘리거나 hyperparameter를 조절하여 model complexity를 줄이는 해결 방안이 있다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___
=> 높은 bias가 문제이다. alpha를 감소시켜야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
    - 평범한 선형 회귀는 과적합 될 수 있으나, 릿지를 사용할 경우 규제를 줌으로써 모델의 과적합을 막을 수 있다.
- 릿지 회귀 대신 라쏘 회귀
    - lasso를 사용함으로써 feature selection을 할 수 있다.
- 라쏘 회귀 대신 엘라스틱넷
    - 특성 수가 훈련 새플 수보다 많거나 특성 몇 개가 강하게 연관되어 있을 때는 엘라스틱을 사용하는 것이 낫다.


### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [9]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [10]:
iris = load_iris()
X = iris['data']
y = iris['target']

# 원-핫 인코딩
encoder = OneHotEncoder(sparse_output=False)
Y = encoder.fit_transform(y.reshape(-1, 1))

# 훈련 세트와 검증 세트로 분할 (8:2 비율)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# 편향을 위해 X에 1 추가
X_train_b = np.c_[np.ones([len(X_train), 1]), X_train]
X_val_b = np.c_[np.ones([len(X_val), 1]), X_val]

In [11]:
def softmax(z):
    # z에서 최댓값을 빼서 오버플로우 방지
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [12]:
n_features = X_train_b.shape[1]
n_classes = Y_train.shape[1]
W = np.random.randn(n_features, n_classes)

learning_rate = 0.1
epochs = 5001
m = len(X_train)

# --- 조기 종료를 위한 설정 ---
patience = 10  # 10번의 epoch 동안 검증 손실이 개선되지 않으면 중단
patience_counter = 0
best_val_cost = np.inf # 가장 좋았던 검증 손실을 저장 (무한대로 초기화)
best_W = None # 최고 성능일 때의 가중치 저장

In [13]:
for epoch in range(epochs):
    # --- 훈련 ---
    z_train = X_train_b.dot(W)
    Y_proba_train = softmax(z_train)
    train_cost = -np.mean(np.sum(Y_train * np.log(Y_proba_train + 1e-8), axis=1))

    gradient = (1/m) * X_train_b.T.dot(Y_proba_train - Y_train)
    W = W - learning_rate * gradient

    # --- 검증 및 조기 종료 확인 ---
    z_val = X_val_b.dot(W)
    Y_proba_val = softmax(z_val)
    val_cost = -np.mean(np.sum(Y_val * np.log(Y_proba_val + 1e-8), axis=1))

    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Train Cost: {train_cost:.4f}, Val Cost: {val_cost:.4f}")

    # 최고 성능 갱신 여부 확인
    if val_cost < best_val_cost:
        best_val_cost = val_cost
        best_W = W.copy() # 현재 가중치를 최고 가중치로 저장
        patience_counter = 0 # 인내심 카운터 초기화
    else:
        patience_counter += 1 # 성능 개선이 없었으므로 카운터 증가

    # 조기 종료 조건 확인
    if patience_counter >= patience:
        print(f"\n조기 종료! (Epoch: {epoch})")
        break

# 최종적으로 가장 성능이 좋았던 가중치를 사용
W = best_W
print("\n학습 완료! 최종 가중치는 검증 손실이 가장 낮았던 시점의 값입니다.")

Epoch 0, Train Cost: 2.8818, Val Cost: 1.7874
Epoch 500, Train Cost: 0.1764, Val Cost: 0.1956
Epoch 1000, Train Cost: 0.1280, Val Cost: 0.1439
Epoch 1500, Train Cost: 0.1081, Val Cost: 0.1227
Epoch 2000, Train Cost: 0.0970, Val Cost: 0.1110
Epoch 2500, Train Cost: 0.0898, Val Cost: 0.1034
Epoch 3000, Train Cost: 0.0848, Val Cost: 0.0981
Epoch 3500, Train Cost: 0.0810, Val Cost: 0.0940
Epoch 4000, Train Cost: 0.0781, Val Cost: 0.0909
Epoch 4500, Train Cost: 0.0758, Val Cost: 0.0883
Epoch 5000, Train Cost: 0.0739, Val Cost: 0.0861

학습 완료! 최종 가중치는 검증 손실이 가장 낮았던 시점의 값입니다.


In [14]:
def predict(X, weights):
    X_b = np.c_[np.ones([len(X), 1]), X]
    z = X_b.dot(weights)
    Y_proba = softmax(z)
    return np.argmax(Y_proba, axis=1)

# 전체 데이터셋으로 정확도 평가
X_b_full = np.c_[np.ones([len(X), 1]), X]
predictions = np.argmax(softmax(X_b_full.dot(W)), axis=1)
accuracy = np.mean(predictions == y)

print(f"\n전체 데이터셋 정확도: {accuracy:.4f}")


전체 데이터셋 정확도: 0.9867
